In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
!pip install opencv-python-headless
!pip install yolov5
!pip install ultralytics --upgrade

In [ ]:
import cv2
import io
import zipfile
import os
import shutil
import numpy as np
import torch
import time
from PIL import Image
from ultralytics import YOLO
from ultralytics.engine.results import Results
from yolov5 import detect
from datetime import datetime
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = '/content/drive/MyDrive/TreinamentoCustomizadoYOLO/obj.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')

extracted_dir = '/content/data/obj'

extracted_dir = '/content/data/obj'
if os.path.exists(extracted_dir) and not os.listdir(extracted_dir):
    os.rmdir(extracted_dir)

In [ ]:
zip_path = '/content/drive/MyDrive/TreinamentoCustomizadoYOLO/valid.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')

extracted_dir = '/content/data/valid'

extracted_dir = '/content/data/valid'
if os.path.exists(extracted_dir) and not os.listdir(extracted_dir):
    os.rmdir(extracted_dir)

In [ ]:
os.makedirs('/content/data/obj/images', exist_ok=True);
os.makedirs('/content/data/obj/labels', exist_ok=True);

os.makedirs('/content/data/valid/images', exist_ok=True);
os.makedirs('/content/data/valid/labels', exist_ok=True);

for file in os.listdir('/content/data/obj'):
    if file.endswith('.jpg'):
        shutil.move(os.path.join('/content/data/obj', file), '/content/data/obj/images')
    elif file.endswith('.txt'):
        shutil.move(os.path.join('/content/data/obj', file), '/content/data/obj/labels')

for file in os.listdir('/content/data/valid'):
    if file.endswith('.jpg'):
        shutil.move(os.path.join('/content/data/valid', file), '/content/data/valid/images')
    elif file.endswith('.txt'):
        shutil.move(os.path.join('/content/data/valid', file), '/content/data/valid/labels')

In [ ]:
!cp /content/data/obj/labels/classes.txt /content/data

In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
pip install -r requirements.txt

In [ ]:
%cd /content/drive/MyDrive/TreinamentoCustomizadoYOLO

In [ ]:
!cp data_custom.yaml /content/data

In [ ]:
!ls /content/data

In [ ]:
%cd /content/yolov5

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/data/data_custom.yaml --weights yolov5s.pt --cache --device 0

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt', force_reload=True)

# Imprimir as classes disponíveis
print("Classes disponíveis:", model.names)

In [ ]:
#captura de foto
def take_photo(filename='photo.jpg', quality=0.5):
    js = Javascript(
        '''
        async function takePhoto(quality) {
            try {
                const stream = await navigator.mediaDevices.getUserMedia({video: true});
                const div = document.createElement('div');
                const capture = document.createElement('button');
                capture.textContent = 'Capture';
                div.appendChild(capture);

                const video = document.createElement('video');
                video.style.display = 'block';
                document.body.appendChild(div);
                div.appendChild(video);
                video.srcObject = stream;
                await video.play();

                google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;

                await new Promise((resolve) => capture.onclick = resolve);

                canvas.getContext('2d').drawImage(video, 0, 0);
                stream.getVideoTracks()[0].stop();
                div.remove();
                return canvas.toDataURL('image/jpeg', quality);
            } catch (error) {
                console.error("Erro ao acessar a câmera:", error);
                alert("Erro ao acessar a câmera ou permissão negada.");
                return null;
            }
        }
        '''
    )
    display(js)
    data = eval_js('takePhoto({})'.format(quality))

    if data is None:
        print("Não foi possível acessar a câmera. Verifique as permissões.")
        return None

    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

#aguarde
time.sleep(1)

image_path = take_photo()
print(f"Image path: {image_path}")  # Debug: Ver se retorna o caminho corretamente

if image_path is None:
    print("Não foi possível tirar a foto. Verifique as permissões da câmera.")
else:
    img = cv2.imread(image_path)

    results = model(img)

    #desenha as caixas delimitadoras
    img_with_boxes = img.copy()
    for *xyxy, conf, cls in results.xyxy[0]:
        label = f'{model.names[int(cls)]} {conf:.2f}'
        xyxy = list(map(int, xyxy))
        cv2.rectangle(img_with_boxes, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
        cv2.putText(img_with_boxes, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    #objetos detectados
    objetos_detectados = [model.names[int(cls)] for cls in results.pred[0][:, -1]]

    #itens para o cotidiano
    itens_dia = {
        'monday': ['backpack', 'book', 'bottle', 'caderno', 'estojo', 'pasta', 'calculadora', 'chaves'],
        'tuesday': ['laptop', 'book', 'book_geografia', 'backpack', 'estojo', 'caderno', 'bottle', 'pasta', 'calculadora', 'chaves'],
        'wednesday': ['backpack', 'book', 'bottle', 'estojo', 'caderno', 'pasta', 'laptop', 'chaves'],
        'thursday': ['laptop', 'book', 'bottle', 'caderno', 'estojo', 'backpack', 'book_geografia', 'chaves'],
        'friday': ['backpack', 'book', 'bottle', 'estojo', 'caderno', 'pasta', 'book_historia', 'chaves'],
    }

    #determina o dia que o usuário se encontra
    dia_atual = datetime.now().strftime('%A').lower()
    objetos_necessarios = itens_dia.get(dia_atual, [])

    #objetos faltantes
    objetos_faltando = [obj for obj in objetos_necessarios if obj not in objetos_detectados]

    #se há objeto faltando
    if objetos_faltando:
        print(f"Atenção! Faltam os seguintes itens: {', '.join(objetos_faltando)}")
    else:
        print("Todos os itens necessários estão presentes!")

    try:
        from google.colab.patches import cv2_imshow
    except ImportError:
        def cv2_imshow(image):
            from PIL import Image
            display(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))

    cv2_imshow(img_with_boxes)

In [ ]:
!ls runs/train/exp/weights

In [ ]:
%cd /content/yolov5

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /path/to/your/test/images

In [ ]:
import matplotlib.pyplot as plt

image_path = 'runs/train/exp/results.png'
img = cv2.imread(image_path)

if img is not None :
    cv2_imshow(img)
else :
    print("Falha ao carregar a imagem.")